# Import Packages

In [1]:
from sw_functions import sub_df, load_main_monsters, create_sw_adjacency_matrix

# Load Cards and Create Adjacency Matrix

In [2]:
MAIN_MONSTERS = load_main_monsters()
SW_ADJACENCY_MATRIX = create_sw_adjacency_matrix(MAIN_MONSTERS) #small world adjacency matrix of all cards

# Find Best Bridges Function

In [47]:
def find_best_bridges(deck_monster_names, required_target_names):
    #inputs: list of monster names and list of monsters that are required to connect with the small world bridges
    #output: The bridges that connect the most cards in your deck and connect with all the required targets
    deck_monster_names = list(set(deck_monster_names) | set(required_target_names)) #union names
    deck_indices = sub_df(MAIN_MONSTERS, deck_monster_names, 'name').index #indices of monsters in deck
    required_indices = sub_df(MAIN_MONSTERS, required_target_names, 'name').index #indices of required connections

    num_required_bridges = len(required_target_names) #number of cards required to connect with one bridge

    required_monster_matrix = SW_ADJACENCY_MATRIX[required_indices,:] #array corresponding to (required connection monsters) by (all monsters)
    num_bridges_to_required_cards = required_monster_matrix.sum(axis=0) #number of required connections satisfied by all monsters
    required_bridge_mask = num_bridges_to_required_cards==num_required_bridges
    bridges = MAIN_MONSTERS[required_bridge_mask].copy() #data frame of monsters connecting all required targets
    required_bridge_indices = bridges.index #indices of monsters that satisfy all required connections
    if len(bridges)==0:
        print('There are no monsters that bridge all required targets.')
        return

    #subset of adjacency matrix corresponding to (deck monsters) by (monsters with connections to the required cards)
    bridge_matrix = SW_ADJACENCY_MATRIX[deck_indices,:][:,required_bridge_indices]

    num_deck_bridges = bridge_matrix.sum(axis=0)
    bridges['number of connections'] = num_deck_bridges
    bridges = bridges[['number of connections', 'name', 'type', 'attribute', 'level', 'atk', 'def']] #reorder columns
    bridges = bridges.sort_values(by=['number of connections','name'], ascending=[False, True]).reset_index(drop=True) #reorder rows
    return bridges

# Example

In [50]:
#cards that we want to find bridges between
deck_monster_names = ['Ash Blossom & Joyous Spring',
                        'Effect Veiler',
                        'Ghost Belle & Haunted Mansion',
                        'Mathmech Addition',
                        'Mathmech Circular',
                        'Mathmech Diameter',
                        'Mathmech Multiplication',
                        'Mathmech Nabla',
                        'Mathmech Sigma',
                        'Mathmech Subtraction',
                        'Parallel eXceed',
                        'Maxx "C"']

#cards that are required to connect with a bridge
required_target_names = ['Mathmech Circular',
                        'Maxx "C"',
                        'Parallel eXceed']

bridges = find_best_bridges(deck_monster_names, required_target_names)
bridges

,number of connections,name,type,attribute,level,atk,def
0,10.0,Boot-Up Corporal - Command Dynamo,Machine,EARTH,4.0,0.0,2000.0
1,10.0,Boot-Up Soldier - Dread Dynamo,Machine,EARTH,4.0,0.0,2000.0
2,10.0,Doshin @Ignister,Cyberse,EARTH,1.0,100.0,800.0
3,10.0,Flowerdino,Dinosaur,EARTH,4.0,2000.0,0.0
4,10.0,Gem-Turtle,Rock,EARTH,4.0,0.0,2000.0
...,...,...,...,...,...,...,...
101,4.0,Rock Band Xenoguitar,Rock,EARTH,4.0,1000.0,2000.0
102,4.0,Sand Moth,Rock,EARTH,4.0,1000.0,2000.0
103,4.0,Scrounging Goblin,Fiend,EARTH,4.0,1000.0,2000.0
104,4.0,Superheavy Samurai Kabuto,Machine,EARTH,4.0,1000.0,2000.0
